In [1]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [2]:
# dij需求i到设施j的成本
d = [[1, 2], [1, 2]]
n = 2   # 两个设施点
m = 2   # 两个需求点
# d = [[1, 2], [3, 4], [5, 6]]
# n = 2   # 两个设施点
# m = 3   # 三个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]


In [3]:
penalty = 40
depth = 5
params = np.ones(depth * 2)

In [4]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

In [5]:
# dtype=np.complex128
def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H


def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    # print(num_qubits)
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2

    # for i in range(m):
    #     Ht = np.zeros((2**num_qubits, 2**num_qubits))
    #     for j in range(n):
    #         Ht += (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2
    #     Ht -= np.eye(2**num_qubits)
    #     Hp += -penalty * Ht @ Ht
    
    # for i in range(m):
    #     for j in range(n):
    #         Ht = (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + m + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, j)) / 2
    #         Hp += -penalty * Ht @ Ht

    return Hp

In [6]:
# def gnrt_Hd(n, m):
#   mn = m * n
#   Hd = np.zeros((2**mn, 2**mn)).astype(np.complex128)
#   for i in range(mn):
#     j = (i + 1) % mn
#     Hd += (add_in_target(mn, i, GateX) @ add_in_target(mn, j, GateX) + add_in_target(mn, i, GateY) @ add_in_target(mn, j, GateY))
#   return -Hd


In [7]:
def gnrt_Hd0():
  Hd = np.zeros((2**3, 2**3)).astype(np.complex128)
  Hd += (add_in_target(3, 0, GateX) @ add_in_target(3, 1, GateX) + add_in_target(3, 0, GateY) @ add_in_target(3, 1, GateY))
  Hd += (add_in_target(3, 1, GateX) @ add_in_target(3, 2, GateX) + add_in_target(3, 1, GateY) @ add_in_target(3, 2, GateY))
  Hd += (add_in_target(3, 2, GateX) @ add_in_target(3, 0, GateX) + add_in_target(3, 2, GateY) @ add_in_target(3, 0, GateY))
  return Hd

In [8]:
def gnrt_Hd1():
  Hd = np.zeros((2**3, 2**3)).astype(np.complex128)
  Hd += (add_in_target(3, 0, GateX) @ add_in_target(3, 1, GateX) + add_in_target(3, 0, GateY) @ add_in_target(3, 1, GateY))
  Hd += (add_in_target(3, 1, GateX) @ add_in_target(3, 2, GateX) - add_in_target(3, 1, GateY) @ add_in_target(3, 2, GateY))
  Hd += (add_in_target(3, 2, GateX) @ add_in_target(3, 0, GateX) - add_in_target(3, 2, GateY) @ add_in_target(3, 0, GateY))
  return Hd

In [9]:
def gnrt_Hd2():
  Hd = np.zeros((2**5, 2**5)).astype(np.complex128)
  Hd += (add_in_target(5, 0, GateX) @ add_in_target(5, 3, GateX) + add_in_target(5, 0, GateY) @ add_in_target(5, 3, GateY))
  Hd += (add_in_target(5, 3, GateX) @ add_in_target(5, 4, GateX) - add_in_target(5, 3, GateY) @ add_in_target(5, 4, GateY))
  Hd += (add_in_target(5, 4, GateX) @ add_in_target(5, 0, GateX) - add_in_target(5, 4, GateY) @ add_in_target(5, 0, GateY))
  return Hd

In [10]:
def gnrt_Hd3():
  Hd = np.zeros((2**6, 2**6)).astype(np.complex128)
  Hd += (add_in_target(6, 0, GateX) @ add_in_target(6, 1, GateX) + add_in_target(6, 0, GateY) @ add_in_target(6, 1, GateY))
  Hd += (add_in_target(6, 1, GateX) @ add_in_target(6, 2, GateX) - add_in_target(6, 1, GateY) @ add_in_target(6, 2, GateY))
  Hd += (add_in_target(6, 2, GateX) @ add_in_target(6, 0, GateX) - add_in_target(6, 2, GateY) @ add_in_target(6, 0, GateY))

  Hd += (add_in_target(6, 1, GateX) @ add_in_target(6, 4, GateX) + add_in_target(6, 0, GateY) @ add_in_target(6, 4, GateY))
  Hd += (add_in_target(6, 4, GateX) @ add_in_target(6, 5, GateX) - add_in_target(6, 4, GateY) @ add_in_target(6, 5, GateY))
  Hd += (add_in_target(6, 5, GateX) @ add_in_target(6, 0, GateX) - add_in_target(6, 5, GateY) @ add_in_target(6, 0, GateY))
  return Hd

In [11]:
a = [1, 2, 3, 4, 5]
a[:3]

[1, 2, 3]

In [12]:
def chg1to3():
  for i in range(16):
    B = np.zeros(16)
    B[i] = 1
    Z = np.zeros((2 ** 4, 2 ** 4)).astype(np.complex128)
    a = add_in_target(4, 0, GateX)@add_in_target(4, 1, GateX)@add_in_target(4, 2, GateX)@add_in_target(4, 3, GateX) + add_in_target(4, 0, GateY)@add_in_target(4, 1, GateY)@add_in_target(4, 2, GateY)@add_in_target(4, 3, GateY)
    b = add_in_target(4, 0, GateX)@add_in_target(4, 1, GateY)@add_in_target(4, 2, GateY)@add_in_target(4, 3, GateX) + add_in_target(4, 0, GateY)@add_in_target(4, 1, GateX)@add_in_target(4, 2, GateX)@add_in_target(4, 3, GateY)
    c = add_in_target(4, 0, GateX)@add_in_target(4, 1, GateY)@add_in_target(4, 2, GateX)@add_in_target(4, 3, GateY) + add_in_target(4, 0, GateY)@add_in_target(4, 1, GateX)@add_in_target(4, 2, GateY)@add_in_target(4, 3, GateX)
    d = add_in_target(4, 0, GateX)@add_in_target(4, 1, GateX)@add_in_target(4, 2, GateY)@add_in_target(4, 3, GateY) + add_in_target(4, 0, GateY)@add_in_target(4, 1, GateY)@add_in_target(4, 2, GateX)@add_in_target(4, 3, GateX)
    Z = a - b + c + d
    return Z
    # print(B.astype(int))
    # print(Z.dot(B).astype(int).real)
    # print("====")

In [13]:
from scipy.linalg import expm
def build_circ(n, m, d, g, params):
  qc = QuantumCircuit(num_qubits)
  beta = params[:depth]
  gamma = params[depth:]
  # for i in range(m * n):
  #   qc.h(i)
  # for i in range(m * n, m * n * 2 - 1):
  #   qc.x(i)
  qc.x(3)
  qc.x(4)
  qc.x(7)
  qc.x(8)
  # for i in range(7, 10):
    # qc.x(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * gamma[dp] * generate_Hp(n, m, d, g)), range(num_qubits), label="Hp") # transpile
    # 
    # for i in range(m * n):
    #   qc.rx(beta[dp], i)
    mov = 2
    qc.unitary(expm(-1j * beta[dp] * chg1to3()/4), range(m * n + 2, 2 * m * n + 2), label="Hd11_1")
    qc.unitary(expm(-1j * beta[dp] * chg1to3()/4), range(m * n - mov, 2 * m * n - mov), label="Hd11_1")
    # qc.unitary(expm(-1j * beta[dp] * gnrt_Hd0()), range(m * n + 3 - mov, 2 * m * n + 2 - mov), label="Hd11_1")
    # qc.unitary(expm(-1j * beta[dp] * gnrt_Hd0()), range(m * n + 3, 2 * m * n + 2), label="Hd11_1_2")
    # qc.unitary(expm(-1j * beta[dp] * gnrt_Hd2()), range(m * n -2 , 2 * m * n - 1), label="Hd11_2")
    # qc.unitary(expm(-1j * beta[dp] * gnrt_Hd3()), range(2 , 8), label="Hd11_3")
    # for i in range(2 * m * n, num_qubits):
    #   qc.rx(beta[dp], i)
  qc.measure_all()
  return qc

In [14]:
print(build_circ(n, m, d, g, np.full(depth * 2, np.pi/4)))

              ┌─────┐           ┌─────┐           ┌─────┐           ┌─────┐»
    q_0: ─────┤0    ├───────────┤0    ├───────────┤0    ├───────────┤0    ├»
              │     │           │     │           │     │           │     │»
    q_1: ─────┤1    ├───────────┤1    ├───────────┤1    ├───────────┤1    ├»
              │     │┌─────────┐│     │┌─────────┐│     │┌─────────┐│     │»
    q_2: ─────┤2    ├┤0        ├┤2    ├┤0        ├┤2    ├┤0        ├┤2    ├»
         ┌───┐│     ││         ││     ││         ││     ││         ││     │»
    q_3: ┤ X ├┤3    ├┤1        ├┤3    ├┤1        ├┤3    ├┤1        ├┤3    ├»
         ├───┤│     ││  Hd11_1 ││     ││  Hd11_1 ││     ││  Hd11_1 ││     │»
    q_4: ┤ X ├┤4    ├┤2        ├┤4    ├┤2        ├┤4    ├┤2        ├┤4    ├»
         └───┘│  Hp ││         ││  Hp ││         ││  Hp ││         ││  Hp │»
    q_5: ─────┤5    ├┤3        ├┤5    ├┤3        ├┤5    ├┤3        ├┤5    ├»
              │     │├─────────┤│     │├─────────┤│     │├─────────┤│     │»

In [15]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]

  # for i in range(m):
  #   t = 0
  #   for j in range(n):
  #     t += num[n * (1 + i) + j]
  #   C += penalty * (t - 1)**2

  # for i in range(m):
  #   for j in range(n):
  #     C += penalty * (num[n * (1 + i) + j] + num[n * (1 + m + i) + j] - num[j]) ** 2
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(n, m, d, g, theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [16]:
cost_function('0000010101')

2

In [17]:
from numpy.lib.utils import source
from scipy.optimize import minimize
import numpy as np
# 初始化迭代计数器
iteration_count = 0
def test(pen, dep, par):
  global penalty, depth, params, iteration_count
  iteration_count = 0
  penalty = pen
  depth = dep
  params = par
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, params, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(n, m, d, g, params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.1f}%".format(counts[x] / shots * 100), cost_function(x))

In [18]:
test(40, 4, np.full(4 * 3, np.pi/3))

Iteration 10, Result: 3.348
Iteration 20, Result: 3.545
Iteration 30, Result: 3.3945
Iteration 40, Result: 3.121
Iteration 50, Result: 3.0115
Iteration 60, Result: 3.0075
Iteration 70, Result: 3.0085
Iteration 80, Result: 3.004
Iteration 90, Result: 3.0015
Iteration 100, Result: 3.001
Iteration 110, Result: 3.001
Iteration 120, Result: 3.001
Iteration 130, Result: 3.001
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 3.001
       x: [ 9.909e-01  1.139e+00  1.070e+00  1.150e+00  1.071e+00
            1.971e+00  2.067e+00  1.114e+00  1.043e+00  1.041e+00
            1.039e+00  1.039e+00]
    nfev: 136
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
0110100100 99.9% 3
0110011000 0.1% 4
1001100100 0.0% 5
